# Programación declarativa @ Academia Ática
# Programación funcional
## Ejercicios preparatorios
## Curso 19-20

# Definiciones auxiliares

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

### Algunas funciones sobre listas

In [ ]:
object Signatures{
    abstract class List[A]{
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def reverse: List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
    }
}

### Definiciones sobre árboles binarios

In [ ]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

In [ ]:
object Tree{
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
    
    def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
        tree match {
            case Empty() => 
                empty
            case Node(left, a, right) =>
                node(foldTree(left)(empty)(node),
                    a,
                    foldTree(right)(empty)(node))
        }
}

import Tree._

### Modelo de datos de películas

In [ ]:
case class MovieDatabase(
    films: Map[Film.Id, Film],
    users: Map[User.Id, User],
    ratings: Map[(Film.Id, User.Id), Rating])
    
case class Film(
    id: Film.Id, 
    title: String, 
    director: String,
    genre: String,
    year: Int,
    country: String)

object Film{
    type Id = Int
}

case class User(
    id: User.Id,
    name: String,
    registered: Int)
        
object User{
    type Id = Int
}
        
case class Rating(
    film: Film.Id,
    user: User.Id,
    score: Int)

In [ ]:
val moviedb: MovieDatabase = MovieDatabase(
    films = Map(
        1 -> Film(1, "Blade Runner", "Ridley Scott", "Sci-Fi", 1982, "United States"),
        2 -> Film(2, "Amanece, que no es poco", "José Luis Cuerda", "Comedy", 1989, "Spain"),
        3 -> Film(3, "El milagro de P. Tinto", "Javier Fesser", "Comedy", 1998, "Spain"),
        4 -> Film(4, "Mars Attacks!", "Tim Burton", "Sci-Fi", 1996, "United States"),
        5 -> Film(5, "2001: A Space Odyssey", "Stanley Kubrick", "Sci-Fi", 1968, "United Kingdom"),
        6 -> Film(6, "El crack Cero", "José Luis Garci", "Film noir", 2019, "Spain"),
        7 -> Film(7, "El crack", "José Luis Garci", "Film noir", 1981, "Spain"),
        8 -> Film(8, "The Maltese Falcon", "John Huston", "Film noir", 1941, "United States"),
        9 -> Film(9, "Chinatown", "Roman Polanski", "Film noir", 1974, "United States"),
        10 -> Film(10, "Batman v. Superman: Dawn of Justice", "Zack Snyder", "Sci-Fi", 2016, "United States"),
        11 -> Film(11, "Dumb and Dumber", "Peter Farrelly", "Comedy", 1994, "United States")
    ),
    users = Map(
        1 -> User(1, "Juan", 1500),
        2 -> User(2, "Alf", 1555),
        3 -> User(3, "Lola", 1644),
        4 -> User(4, "Lola", 1655),
        5 -> User(5, "Dinu", 1622)),
    ratings = Map(
        (1,1) -> Rating(1,1,5),
        (1,2) -> Rating(1,2,1),
        (1,3) -> Rating(1,3,4),
        (1,4) -> Rating(1,4,3),
        (2,1) -> Rating(2,1,1),
        (2,4) -> Rating(2,4,1),
        (4,1) -> Rating(4,1,3),
        (5,4) -> Rating(5,4,2),
        (6,1) -> Rating(6,1,2),
        (7,1) -> Rating(7,1,3),
        (7,2) -> Rating(7,2,3),
        (7,3) -> Rating(7,3,3),
        (8,2) -> Rating(8,2,2),
        (9,1) -> Rating(9,1,1),
        (10,1) -> Rating(10,1,0),
        (10,3) -> Rating(10,3,0),
        (11,1) -> Rating(11,1,0),
        (11,2) -> Rating(11,2,1),
        (11,4) -> Rating(11,4,2)))

In [ ]:
object BasicQueries{
    
    // Entities
    
    def films(mdb: MovieDatabase): List[Film] =
        mdb.films.values.toList
    
    def filmIds(mdb: MovieDatabase): List[Film.Id] =
        mdb.films.keys.toList

    def getFilm(id: Film.Id)(mdb: MovieDatabase): List[Film] = 
        mdb.films.get(id).toList
    
    def userIds(mdb: MovieDatabase): List[User.Id] = 
        mdb.users.keys.toList
    
    def getUser(id: User.Id)(mdb: MovieDatabase): List[User] = 
        mdb.users.get(id).toList
    
    // 1-N relationships
    
    def films(dir: String)(mdb: MovieDatabase): List[Film.Id] = 
        mdb.films.filter(_._2.director == dir).map(_._1).toList
    
    // N-M relationships
    
    def ratings(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.values.toList
    
    def userRatings(user: User.Id)(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.filter(_._1._2 == user).values.toList
    
    def filmRatings(film: Film.Id)(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.filter(_._1._1 == film).values.toList
}

import BasicQueries._

# Árboles

Implementa las funciones cuyo propósito se describe a continuación usando la función `foldTree` o recursividad, si no es posible el uso de `foldTree`.

#### 1. Una función que dado un árbol y un elemento, determine si éste último está en una hoja.

In [ ]:
class TestLeafContains(
    leafContains: Tree[Int] => Int => Boolean
) extends FlatSpec with Matchers {
    "leafContains" should "work" in {
        leafContains(void)(1) shouldBe false
        leafContains(leaf(1))(1) shouldBe true
        leafContains(left(leaf(1), 2))(2) shouldBe false
        leafContains(node(leaf(1), 2, leaf(3)))(3) shouldBe true
        leafContains(node(leaf(1), 2, leaf(3)))(7) shouldBe false
    }
}

In [ ]:
def leafContains[A](tree: Tree[A])(elem: A): Boolean = 
    ???

In [ ]:
run(new TestLeafContains(leafContains))

#### 2. Una función que dado un árbol y dos números enteros, correspondientes a dos niveles del árbol, devuelva una lista con los elementos situados entre los dos niveles. El primer nivel de un árbol, en el que está situada su raíz, es el 1. Si el primer nivel es superior al segundo nivel, debe devolver una lista vacía. Si no existen ambos niveles, debe devolver una lista vacía. Si el primer nivel existe pero, el segundo no, debe devolver todos los elementos que haya desde el primer nivel hasta el final del árbol.

In [ ]:
class TestElementsBetweenLevels(
    elementsBetweenLevels: Tree[Int] => (Int, Int) => List[Int]
) extends FlatSpec with Matchers {
    "elementsBetweenLevels" should "work" in {
        elementsBetweenLevels(void)(5, 2) shouldBe List()
        elementsBetweenLevels(node(leaf(1), 8, leaf(-4)))(3, 4) shouldBe List()
        elementsBetweenLevels(node(leaf(1), 8, leaf(-4)))(2, 5) shouldBe List(1, -4)
        elementsBetweenLevels(left(node(leaf(1), 8, leaf(-4)), 5))(2, 2) shouldBe List(1, -4)
        elementsBetweenLevels(left(node(leaf(1), 8, leaf(-4)), 5))(1, 3) shouldBe left(node(leaf(1), 8, leaf(-4)), 5)
    }
}

In [ ]:
def elementsBetweenLevels[A](tree: Tree[A])(level1: Int, level2: Int): List[A] = 
    ???

In [ ]:
run(new TestElementsBetweenLevels(elementsBetweenLevels))

#### 3. Una función que dado un árbol devuelva uno nuevo habiendo eliminado las hojas del original.

In [ ]:
class TestRemoveLeaves(
    removeLeaves: Tree[Char] => Tree[Char]
) extends FlatSpec with Matchers {
    "removeLeaves" should "work" in {
        removeLeaves(void) shouldBe void
        removeLeaves(leaf('*')) shouldBe void
        removeLeaves(left(leaf('h'), '?')) shouldBe leaf('?')
        removeLeaves(left(right(';', leaf('a')), '2')) shouldBe left(leaf(';'), '2')
        removeLeaves(node(right(';', leaf('a')), '2', leaf('-'))) shouldBe left(leaf(';'), '2')
    }
}

In [ ]:
def removeLeaves[A](tree: Tree[A]): Tree[A] =
    ???

In [ ]:
run(new TestRemoveLeaves(removeLeaves))

#### 4. Una función que dado un árbol determine si es un zurdo. Un árbol es zurdo si se da alguna de las siguientes condiciones:
- Es un árbol vacío
- Es un nodo hoja
- Sus hijos izquierdo y derecho son zurdos y más de la mitad de sus
descendientes están en el hijo izquierdo

In [ ]:
class TestIsLeftHanded(
    isLeftHanded: Tree[Int] => Boolean
) extends FlatSpec with Matchers {
    "isLeftHanded" should "work" in {
        isLeftHanded(void) shouldBe true
        isLeftHanded(leaf(2)) shouldBe true
        isLeftHanded(node(leaf(1), 0, leaf(7))) shouldBe false
        isLeftHanded(node(left(leaf(8), 1), 0, leaf(7))) shouldBe true
        isLeftHanded(node(leaf(4), 7, node(leaf(6), 14, leaf(20)))) shouldBe false
    }
}

In [ ]:
def isLeftHanded[A](tree: Tree[A]): Boolean =
    ???

In [ ]:
run(new TestRemoveLeaves(removeLeaves))

#### 5. Una función que dado un árbol devuelva el número de nodos que tienen vacío, solamente, su hijo derecho.

In [ ]:
class TestNumEmptyRightSon(
    numEmptyRightSon: Tree[Int] => Int
) extends FlatSpec with Matchers {
    "numEmptyRightSon" should "work" in {
        numEmptyRightSon(void) shouldBe 0
        numEmptyRightSon(leaf(25)) shouldBe 0
        numEmptyRightSon(left(leaf(1), 4)) shouldBe 1
        numEmptyRightSon(node(leaf(-2), 9, leaf(10))) shouldBe 0
    }
}

In [ ]:
def numEmptyRightSon[A](tree: Tree[A]): Int =
    ???

In [ ]:
run(new TestNumEmptyRightSon(numEmptyRightSon))

# Listas

Implementa las funciones cuyo propósito se describe a continuación, utilizando para ello las herramientas que consideres oportunas, es decir, funciones de orden superior como `foldLeft`, `foldRight`, `filter`, `map`, etcétera, o recursividad.

#### 1. Una función que dada una lista de números enteros y un número entero n, devuelva una lista con los n elementos mayores de la lista original. A tener en cuenta:
- La función `min` de listas devuelve el menor de sus elementos.

- La función `diff` permite calcular la diferencia entre 2 listas: `List(1, 2, 3) diff List(3, 1) = List(2)`.

In [ ]:
class TestNBigger(
    nBigger: List[Int] => Int => List[Int]
) extends FlatSpec with Matchers{
    "nBigger" should "work" in {
        nBigger(List(8, 4, -5, 6, -1, 0, 2, 6, -10, 7))(4) shouldBe List(8, 6, 6, 7)
        nBigger(List(8, 4, -5, 6, -1, 0, 2, 6, -10, 7))(7) shouldBe List(8, 4, 6, 0, 2, 6, 7)
        nBigger(List(8, 4, -5, 6, -1, 0, 2, 6, -10, 7))(11) shouldBe List(8, 4, -5, 6, -1, 0, 2, 6, -10, 7)
    }
}

In [ ]:
def nBigger(list: List[Int])(n: Int): List[Int] = 
    ???

In [ ]:
run(new TestNBigger(nBigger))

#### 2. Una función que reciba dos listas y determine si la primera está contenida en la segunda. Se dice que una lista está contenida en otra, si los elementos de la primera aparecen dentro de la segunda, en el mismo orden y de forma consecutiva.

In [ ]:
class TestIsContain(
    isContain: List[Int] => List[Int] => Boolean
) extends FlatSpec with Matchers{
    "isContain" should "work" in {
        isContain(List())(List(4, 5)) shouldBe true
        isContain(List(4, 4, 2))(List(5, 4, 4, 5, 2, 9)) shouldBe false
        isContain(List(4, 4, 2))(List(5, 4, 4, 5, 4, 4, 2, 9)) shouldBe true
        isContain(List(4, 5))(List()) shouldBe false
    }
}

In [ ]:
def isContain[A](list1: List[A])(list2: List[A]): Boolean =
    ???

In [ ]:
run(new TestIsContain(isContain))

#### 3. Una función que reciba una lista de listas y sea capaz de invertir los elementos de las sublistas y, a su vez, de la lista.

In [ ]:
class TestDeepReverse(
    deepReverse: List[List[Int]] => List[List[Int]]
) extends FlatSpec with Matchers{
    "deepReverse" should "work" in {
        deepReverse(List(List(1, 2, 3), List(3, 4, 5))) shouldBe List(List(5, 4, 3), List(3, 2, 1))
    }
}

In [ ]:
def deepReverse[A](list: List[List[A]]): List[List[A]] =
    ???

In [ ]:
run(new TestIsContain(isContain))

# Ejercicio Base de Datos

Dado el modelo de datos de películas, implementar una consulta que devuelva los títulos de las películas que hayan sido valoradas en, al menos, 2 ocasiones y, además, su fecha de estreno sea en el siglo XXI.

In [ ]:
class TestRecentMovies(
    recentMovies: MovieDatabase => List[String]
) extends FlatSpec with Matchers {
    "recentMovies" should "work" in {
        recentMovies(moviedb) shouldBe List()
    }
}

In [ ]:
def recentMovies(mdb: MovieDatabase): List[String] =
    ???

In [ ]:
run(new TestRecentMovies(recentMovies))